<a href="https://colab.research.google.com/github/FarahKandil/Data-Analysis/blob/main/Copy_of_Final_of_Block_Ciphers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Table of contents:

* [Introduction to block ciphers](#intro-block)
* [Padding a message](#message-padding)
* [The Advanced Encryption Standard (AES)](#AES)
* [Modes of operation of block ciphers](#modes)
* [Size of the output ciphertex on AES](#size)
* [Bonus: Fernet cipher](#fernet)
    
Author: [Sebastià Agramunt Puig](https://github.com/sebastiaagramunt) for [OpenMined](https://www.openmined.org/) Privacy ML Series course.



## Block Ciphers <a class="anchor" id="intro-block"></a>

Block ciphers as opposed to stream ciphers take a block of the plaintext (a specific amount of bytes) and encrypts it into a block with the same size. In this section we will use the Advanced Encryption Standard (AES) to understand block ciphers. In the next schema it is shown how an original message of arbitrary $N$ bytes is converted into a ciphertext having blocks of $K$ bytes. The ciphertext size is always a multiple of $K$ bytes.

<img src="img/block_cipher.png" style="width:1100px"/>

## Padding a message <a class="anchor" id="message-padding"></a>

Most of the times the lenght of the message is not a multiple of the block size so we need to "pad" the message to have the required length. A common padding function is [PKCS7](https://en.wikipedia.org/wiki/Padding_(cryptography)). Basically what PKCS7 does is appendinng a list of bytes with the same value corresponding to the number of bytes needed to complete the block.




## Encrypting using AES (Advanced Encryption Standard) <a class="anchor" id="AES"></a>

AES is a block cipher that was established as a standard by NIST in 2001 (after a public call to improve/substitute DES encryption algorithm in 1997). AES is a subset of the Rijndael block cipher developed by Vincent Rijmen and Joan Daemen submitted to NIST during the [AES selection process](https://en.wikipedia.org/wiki/Advanced_Encryption_Standard_process).


We are not going to go into the details of te exact implementation but the readers are referred to the book of [Katz and Lindell](http://www.cs.umd.edu/~jkatz/imc.html) Chapter 6 section 2. Also Mike Pound explains AES in this [video](https://www.youtube.com/watch?v=O4xNJsjtN6E&t=524s&ab_channel=Computerphile), check it out!

In [ ]:
from typing import List, Union, Tuple, Any, Generator
from itertools import chain
import warnings
import os

ECB = 1
CBC = 2

In [ ]:
S_box = [
    [0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76],
    [0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0],
    [0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15],
    [0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75],
    [0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84],
    [0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf],
    [0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8],
    [0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2],
    [0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73],
    [0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb],
    [0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79],
    [0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08],
    [0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a],
    [0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e],
    [0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf],
    [0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16]
]
Rcon = (
    0x01, 0x02, 0x04, 0x08, 0x10, 0x20,
    0x40, 0x80, 0x1b, 0x36, 0x6c, 0xd8,
    0xab, 0x4d, 0x9a, 0x2f, 0x5e, 0xbc,
    0x63, 0xc6, 0x97, 0x35, 0x6a, 0xd4,
    0xb3, 0x7d, 0xfa, 0xef, 0xc5, 0x91,
)

In [ ]:
def x_time(a): return (((a << 1) ^ 0x1B) & 0xFF) if (a & 0x80) else (a << 1)

In [ ]:
#splits a list
def split_function(a: List[bytes], n: int) -> List[List[bytes]]:
   
    k, m = divmod(len(a), n)
    return list(a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))

In [ ]:
#makes the list into blocks
def block(l: List[Any], n: int) -> Generator:
    
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [ ]:
#carries out confusion when expanding keys
def con(block: List[int], rc: bytes) -> List[bytes]:

    block = [__sub_byte(b, S_box) for b in block[1:] + [block[0]]]
    return [block[0] ^ rc] + block[1:]

In [ ]:
#substitution from s-box
def __sub_byte(b: int, box: List[List[bytes]]) -> bytes:
   
    b = hex(b)[2:]
    if len(b) == 1:
        b = '0' + b
    row, col = list(b)
    return box[int(row, 16)][int(col, 16)]

In [ ]:
def _sub_bytes(state, box):
    new_mat = []
    for row in state:
        new_row = []
        for v in row:
            new_row.append(__sub_byte(v, box))
        new_mat.append(new_row)
    return new_mat

In [ ]:
def shift_rows(s: List[List[bytes]]) -> List[List[bytes]]:
  
    s[0][1], s[1][1], s[2][1], s[3][1] = s[1][1], s[2][1], s[3][1], s[0][1]
    s[0][2], s[1][2], s[2][2], s[3][2] = s[2][2], s[3][2], s[0][2], s[1][2]
    s[0][3], s[1][3], s[2][3], s[3][3] = s[3][3], s[0][3], s[1][3], s[2][3]
    return s




In [ ]:
def round(state: List[List[bytes]], round_key: List[Union[List[List[int]], List[list]]]) -> List[List[int]]:
  
    state = _sub_bytes(state, S_box)
    state = shift_rows(state)
    state = mix_columns(state)
    state = _add_round_key(state, round_key)
    return state


def mix_column(col: List[bytes]) -> List[bytes]:
    
    t = col[0] ^ col[1] ^ col[2] ^ col[3]
    u = col[0]
    col[0] ^= t ^ x_time(col[0] ^ col[1])
    col[1] ^= t ^ x_time(col[1] ^ col[2])
    col[2] ^= t ^ x_time(col[2] ^ col[3])
    col[3] ^= t ^ x_time(col[3] ^ u)
    return col


def mix_columns(state:  List[List[bytes]]) -> list:
   
    return [mix_column(column) for column in state]



def _add_round_key(state: List[List[bytes]], round_key: List[Union[List[List[int]], List[list]]]) -> list:
  
    new_state = []
    for r1, r2 in zip(state, round_key):
        new_col = []
        for v1, v2 in zip(r1, r2):
            new_col.append(v1 ^ v2)
        new_state.append(new_col)
    return new_state


def _pad_data(data: bytes, n: int = 16) -> bytes:
   
    pad_len = n - (len(data) % n)
    return data + bytes([pad_len] * pad_len)


NUM_ROUNDS = {16: 10, 24: 12, 32: 14}
NUM_WORDS = {16: 4, 24: 6, 32: 8}



class AES:
    def __init__(self, key: bytes, mode=CBC):
        if len(key) not in NUM_ROUNDS:
            raise ValueError("Only 128, 192 and 256 bit keys are supported!")

        if mode != CBC and mode != ECB:
            raise ValueError("Unsupported mode!")

        self.nb = 4
        self.nk = NUM_WORDS[len(key)]
        self.nr = NUM_ROUNDS[len(key)]
        self.mode = mode
        self.block_length = 16
        self.round_keys = self._expand_key(key)

    def _expand_key(self, key: bytes) -> List[Tuple[Any]]:
       
        w = []
        for i in range(self.nk):
            w.append([key[4 * i], key[4 * i + 1], key[4 * i + 2], key[4 * i + 3]])

        for i in range(self.nk, (self.nb * (self.nr + 1))):
            tmp = w[i - 1]
            if i % self.nk == 0:
                tmp = con(tmp, Rcon[int(i / self.nk) - 1])
            elif self.nk > 6 and i % self.nk == self.nb:
                tmp = _sub_bytes([tmp], S_box)[0]
            w.append([x ^ y for x, y in zip(w[i - self.nk], tmp)])
        return list(zip(*[iter(w)] * self.nb))

    def encrypt(self, data: bytes, iv=None) -> tuple:
        
        if self.mode == ECB and iv is not None:
            warnings.warn("error")

        if self.mode == CBC and iv is None:
            iv = os.urandom(self.block_length)

        state = _pad_data(data)
        blocks = list(block(list(state), self.block_length))

        cipher = self._encrypt_CBC(blocks, iv) if self.mode == CBC else self._encrypt_ECB(blocks)

        return cipher, iv

    def _encrypt_ECB(self, blocks: List[bytes]) -> bytes:
        
        return b''.join([self._encrypt_single_block(block) for block in blocks])

    def _encrypt_CBC(self, blocks: List[bytes], iv: bytes) -> bytes:
       
        encrypted_blocks = [iv]
        for block, prev in zip(blocks, encrypted_blocks):
            next_block = bytes([x ^ y for x, y in zip(block, prev)])
            encrypted_blocks.append(self._encrypt_single_block(next_block))

        return b''.join(encrypted_blocks[1:])

    def _encrypt_single_block(self, data: bytes) -> bytes:
       
        state = split_function(list(data), 4)
        state = _add_round_key(state, self.round_keys[0])

        for i in range(1, self.nr):
            state = round(state, self.round_keys[i])

        state = _sub_bytes(state, S_box)
        state = shift_rows(state)
        state = _add_round_key(state, self.round_keys[-1])

        state = bytes(list(chain(*state)))

        return state

    
  

In [ ]:
def _encrypt_ECB(self, blocks: List[bytes]) -> bytes:
        
        return b''.join([self._encrypt_single_block(block) for block in blocks])

   

In [ ]:
key = os.urandom(16)
aes= AES(key)
var=input('Please enter the plaintext').encode('ASCII')
cipher_text, iv = aes.encrypt(var)
print("This is the cipher text after encryption",cipher_text)

Please enter the plaintexthello
This is the cipher text after encryption b'2n\x85\xcfO\xc0\xef\xf4\xcdR\xff]\x81\x9d^\xa8'


## Modes of operation of block ciphers <a class="anchor" id="mode"></a>

A block cipher by itself is only suitable for the secure cryptographic transformation (encryption or decryption) of one fixed-length group of bits called a block. A mode of operation describes how to repeatedly apply a cipher's single-block operation to securely transform amounts of data larger than a block ([Wikipedia](https://en.wikipedia.org/wiki/Block_cipher_mode_of_operation)).

The first mode is "not doing anything", this is the Electronic Codebook mode. See the figure below (from Wikipedia).

<img src="img/ECB_mode.png" style="width:1100px"/>

We are lucky and in ```cryptography``` package ECB implemented in ```cryptography.hazmat.primitives.ciphers.ECB``` function (we've seen in the previous example!).

Now we can encrypt the same message twice and see what we get in the ciphertext:

This is not a desirable outcome. If I want to send the same message twice, I really don't want to send the same ciphertext. What if in all comunications I start by "Dear..." and the attacker knows it?. A better mode is the Cipher block chaining (CBC):

<img src="img/CBC_mode.png" style="width:1100px"/>

In this case we take a random initialization vector and perform XOR operation with the block of plaintext, then we feed this into the encryptor, after that we obtain the ciphertext. This ciphertext is used as the initialization vector to encrypt the next block.

In [ ]:
def non_encoder(block, key):
    """A basic encoder that doesn't actually do anything"""
    return enc(block, len(key))

In [ ]:
def xor_encoder(block, key):
    block = enc(block, len(key))
    cipher = [b ^ k for b, k in zip(block, key)]
    return cipher

In [ ]:
def bits2a(b):
     return ''.join(chr(int(''.join(x), 2)) for x in zip(*[iter(b)]*8))

In [ ]:
def aes_encoder(block, key):
   # block = enc(block, len(key))
    block=encrypt(self, data: bytes, iv=None)
    # the pycrypto library expects the key and block in 8 bit ascii 
    # encoded strings so we have to convert from the bit string
    block = bits2a(block)
    key = bits2a(key)
    ecb = AES.new(key, AES.MODE_ECB)
    return bits2a(ecb.encrypt(block))

In [ ]:
def cipher_block_chaining(plaintext, key, init_vec, block_size, block_enc):
   cipher = []
   cipher.extend(init_vec)
   for i in range(len(plaintext) / block_size + 1):
        start = i * block_size
        if start >= len(plaintext):
            break
        end = min(len(plaintext), (i+1) * block_size)
        block = plaintext[start:end]
        pre = cipher[start:end]
        m = [int(block[j] != pre[j]) for j in range(len(pre))]
        
        cipher.extend(block_enc(m, key))
        return cipher[block_size:]

        message = input("Enter your Message for AES Encryption:- ")

# Print Encrypted Message 
print("\nEncryption of Message Using AES:-", cipher[block_size:].text)
print("Hex of Cipher Text:-", cipher_text.hex())



In [ ]:

import os
def electronic_cookbook(plaintext):
  block_size=128
  key = os.urandom(16)
  block_enc=aes(key)
    """Return the ecb encoding of `plaintext"""
    if(plaintext.length()%16!=0)
       enc(key, plaintext)
      
    cipher = []
    # break the plaintext into blocks
    # and encode each one
    for i in range(len(plaintext) / block_size + 1):
        start = i * block_size
        if start >= len(plaintext):
            break
        end = min(len(plaintext), (i+1) * block_size)
        block = plaintext[start:end]
        cipher.extend(block_enc(block, key))
    return cipher

## Bonus: Fernet <a class="anchor" id="fernet"></a>

Another block cipher implemented in cryptography package is [Fernet](https://asecuritysite.com/encryption/fernet). 

In [ ]:
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend

import sys
import binascii
import base64

def get_key(password):
    digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
    digest.update(password)
    return base64.urlsafe_b64encode(digest.finalize())

if (len(sys.argv)>1):
	val=sys.argv[1]

if (len(sys.argv)>2):
	password=str(sys.argv[2])

if (len(password)>1):
	key = get_key(password.encode())
else:
	key = Fernet.generate_key()

val = input("Enter your Message :- ")
password = val.split(' ')[0]

print("\nMessage:-", val)
print("Key: ",binascii.hexlify(bytearray(key)))


cipher_suite = Fernet(key)
cipher_text = cipher_suite.encrypt(val.encode())
cipher=binascii.hexlify(bytearray(cipher_text))
print("Cipher: ",cipher)

print("\nVersion:\t",cipher[0:2])
print("Time stamp:\t",cipher[2:18])
print("IV:\t\t",cipher[18:50])
print("HMAC:\t\t",cipher[-64:])

plain_text = cipher_suite.decrypt(cipher_text)
print("\nPlain text: ",plain_text)
print("password",password)


Enter your Message :- hello

Message:- hello
Key:  b'3372307430767676766f5a44636c35662d517657644444746e655a366144416e793636616b714b6c4478733d'
Cipher:  b'674141414141426a67533346753059636b70704d673264424d3567694b4e6c384858584f635859507179704c674d635366497a766d7a436a4b6c686556305f305955486a4171314d554f437a74496954434e504e4f4e6d7763723362517a6d7442513d3d'

Version:	 b'67'
Time stamp:	 b'4141414141426a67'
IV:		 b'533346753059636b70704d673264424d'
HMAC:		 b'4171314d554f437a74496954434e504e4f4e6d7763723362517a6d7442513d3d'

Plain text:  b'hello'
password hello


In [ ]:
! pip install pycryptodome
! pip install cryptography
! pip install Crypto
from Crypto.Util.Padding import pad
from base64 import b64encode
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
import cryptography
import json
import base64

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.3 MB 5.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.1 MB 5.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 587 kB 5.2 MB/s 
